In [4]:
from sklearn.datasets import fetch_20newsgroups
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]
fetch_subset = lambda subset: fetch_20newsgroups(
    subset=subset, categories=categories,
    shuffle=True, random_state=42,
    remove=('headers', 'footers', 'quotes'))
train = fetch_subset('train')
test = fetch_subset('test')

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()
clf = LogisticRegressionCV()
pipeline = Pipeline([('vec', vec), ('clf', clf)])
pipeline.fit(train['data'], train['target'])

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
   ...2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0))])

In [6]:
from eli5.sklearn import explain_weights, explain_prediction
from eli5.formatters import format_as_html, format_as_text, format_html_styles

print(format_as_text(explain_weights(clf, vec, target_names=train['target_names'])))

Explained as: linear model

Features with largest coefficients per class.
Caveats:
1. Be careful with features which are not
   independent - weights don't show their importance.
2. If scale of input features is different then scale of coefficients
   will also be different, making direct comparison between coefficient values
   incorrect.
3. Depending on regularization, rare features sometimes may have high
   coefficients; this doesn't mean they contribute much to the
   classification result for most examples.

y='alt.atheism' top features
----------------------
     atheism  +18.067
    atheists  +16.546
    religion  +14.379
       bobby  +14.378
     matthew  +14.321
       motto  +13.380
     atheist  +13.209
       islam  +13.021
       nanci  +12.787
enviroleague  +12.247
       loans  +12.100
       satan  +11.656
     posting  +11.482
enlightening  +11.189
     natural  +11.092
         ...   (6379 more positive features)
         ...   (20481 more negative features)
       

In [7]:
from IPython.core.display import display, HTML
show_html = lambda html: display(HTML(html))
show_html_expl = lambda expl, **kwargs: show_html(format_as_html(expl, include_styles=False, **kwargs))
show_html(format_html_styles())

In [24]:
show_html_expl(explain_weights(clf, vec, target_names=train['target_names']))

Feature,Weight
atheism,+18.067
atheists,+16.546
religion,+14.379
bobby,+14.378
matthew,+14.321
motto,+13.380
atheist,+13.209
islam,+13.021
nanci,+12.787
enviroleague,+12.247


In [23]:
show_html_expl(explain_prediction(clf, test['data'][2], vec, target_names=train['target_names']), force_weights=True)

Feature,Weight
some,+0.886
much,+0.537
…,6 more positive
…,15 more negative
he,-0.405
how,-0.407
am,-0.432
from,-0.461
features,-0.509
where,-0.607


In [15]:
show_html_expl(explain_prediction(clf, test['data'][4], vec, target_names=train['target_names']), force_weights=False)

In [25]:
import numpy as np
for doc in test['data'][:10]:
    expl = explain_prediction(clf, doc, vec, target_names=train['target_names'])
    # haaack - leave only the winner
    max_class_idx = np.argmax([cl['proba'] for cl in expl['classes']])
    expl['classes'] = [expl['classes'][max_class_idx]]
    show_html_expl(expl, force_weights=False)